<a href="https://colab.research.google.com/github/shima-aflatounian/Sentiment_Analysis/blob/main/svm_1_to_1_classifier_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

including neutral tweets
1 to 1 classification: nr of required classifiers : n* (n-1)/2
we have 3 class +,-, neutral > n=3 > required nr of classifiers: 3
* voting approach for predict the class of test tweet: the class with most majority vote

voting section inspired by:
https://www.researchgate.net/publication/318484311_A_Study_using_Support_Vector_Machines_to_Classify_the_Sentiments_of_Tweets

In [ ]:
import numpy as np
import pandas as pd
import csv
import pickle
import os.path
from time import time
import re
from nltk.stem import PorterStemmer
from sklearn.svm import LinearSVC
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import tensorflow as tf
!pip install emoji
import emoji

def get_words_in_tweets(tweets):
    all_words = []
    for (words, sentiment) in tweets:
        all_words.extend(words)
    return all_words
def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

def extract_features1(document):
    document_words = set(document)
    features = {}
    for word in word_features1:
        features['contains(%s)' % word] = (word in document_words)
    return features

def extract_features2(document):
    document_words = set(document)
    features = {}
    for word in word_features2:
        features['contains(%s)' % word] = (word in document_words)
    return features

def extract_features3(document):
    document_words = set(document)
    features = {}
    for word in word_features3:
        features['contains(%s)' % word] = (word in document_words)
    return features

def process_tweet(tweet):
    new_tweet = tweet.lower()
    new_tweet = re.sub(r'@\w+', '', new_tweet) # Remove @s
    new_tweet = re.sub(r'#', '', new_tweet) # Remove hashtags
    new_tweet = re.sub(r':', ' ', emoji.demojize(new_tweet)) # Turn emojis into words
    new_tweet = re.sub(r'http\S+', 'url',new_tweet) # Remove URLs
    new_tweet = re.sub(r'\$\S+', 'dollar', new_tweet) # Change dollar amounts to dollar
    new_tweet = re.sub(r'[^a-z0-9\s]', '', new_tweet) # Remove punctuation
    new_tweet = re.sub(r'[0-9]+', 'number', new_tweet) # Change number values to number
    new_tweet = new_tweet.split(" ")
    new_tweet = list(map(lambda x: ps.stem(x), new_tweet)) # Stemming the words
    new_tweet = list(map(lambda x: x.strip(), new_tweet)) # Stripping whitespace from the words
    if '' in new_tweet:
        new_tweet.remove('')
    return new_tweet

data = pd.read_csv("https://raw.githubusercontent.com/gabriel-david-orozco/Frequency_words_Spark_vs_Sequential_Python/main/Tweets.csv")


confidence_threshold = 0.7
train_percentage = 0.9
test_percentage = 1-train_percentage
ngbias = 2500   #we introduced a number of negative tweets to remove from the database (as there is a bias to giving negative opinions in Twitter)

data = data.drop(data.query("airline_sentiment_confidence < @confidence_threshold").index, axis=0).reset_index(drop=True)
tweets_df = pd.concat([data['text'], data['airline_sentiment']], axis=1)
tweets_df.isna().sum().sum()
tweets_df['airline_sentiment'].value_counts()
sentiment_ordering = ['negative', 'neutral', 'positive']

tweets_df['airline_sentiment'] = tweets_df['airline_sentiment'].apply(lambda x: sentiment_ordering.index(x))

ps = PorterStemmer()
tweets1 = []  #this list will contain the training set of the positive vs negative classifier
tweets2 = []  #this list will contain the training set of the neutral vs negative classifier
tweets3 = []  #this list will contain the training set of the positive vs neutral classifier

ttweets1 = []  #this list will contain the testing set of the positive vs negative classifier
ttweets2 = []  #this list will contain the testing set of the neutral vs negative classifier
ttweets3 = []  #this list will contain the testing set of the positive vs neutral classifier

#file = open("lexicon.tff")
#lexicon = file.readlines()
#unique_words = []
#for line in lexicon:
#    if(line.split(" ")[4][9:]=="y"):
#        if(line.split(" ")[2][6:] not in unique_words):
#            unique_words.append(line.split(" ")[2][6:])
#            if(line.split(" ")[5][14:-1] == "neutral"):
#                tweets2.append(((line.split(" ")[2][6:]), line.split(" ")[5][14:-1]))
#                tweets3.append(((line.split(" ")[2][6:]), line.split(" ")[5][14:-1]))
#            if(line.split(" ")[5][14:-1] == "positive"):
#                tweets1.append(((line.split(" ")[2][6:]), line.split(" ")[5][14:-1]))
#                tweets3.append(((line.split(" ")[2][6:]), line.split(" ")[5][14:-1]))
#            if(line.split(" ")[5][14:-1] == "negative"):
#                tweets1.append(((line.split(" ")[2][6:]), line.split(" ")[5][14:-1]))
#                tweets2.append(((line.split(" ")[2][6:]), line.split(" ")[5][14:-1]))

new_data = pd.read_csv("https://raw.githubusercontent.com/gabriel-david-orozco/Frequency_words_Spark_vs_Sequential_Python/main/new_dataset.tsv", sep='\t')

full_tweets = tweets_df['text']
#full_tweets = pd.concat([tweets_df['text'],new_data['TWEETS']]) # the process has not done on text i guess? full_tweets=process_tweet(tweets_df['text])
sentiment = tweets_df['airline_sentiment']
#sentiment = pd.concat([tweets_df['airline_sentiment'], new_data['SENTIMENTS']])
test_tweets = new_data['TWEETS']
test_sentiment = new_data['SENTIMENTS']
ngcount = 0
for s in range(1, len(full_tweets)):
    if sentiment[s] == 2:
        tweets1.append((process_tweet(full_tweets[s]), 'positive'))
    elif sentiment[s] == 0:
        ngcount=ngcount+1
        if ngcount > ngbias:
            tweets1.append((process_tweet(full_tweets[s]), 'negative'))
ngcount = 0
for s in range(1, len(full_tweets)):
    if sentiment[s] == 1:
        tweets2.append((process_tweet(full_tweets[s]), 'neutral'))
    elif sentiment[s] == 0:
        ngcount=ngcount+1
        if ngcount > ngbias:
            tweets2.append((process_tweet(full_tweets[s]), 'negative'))

for s in range(1, len(full_tweets)):
    if sentiment[s] == 2:
        tweets3.append((process_tweet(full_tweets[s]), 'positive'))
    elif sentiment[s] == 1:
        tweets3.append((process_tweet(full_tweets[s]), 'neutral'))





for s in range(1, len(test_tweets)):
    if test_sentiment[s] == 'negative':
        ttweets1.append((process_tweet(test_tweets[s]), 'negative'))
        ttweets2.append((process_tweet(test_tweets[s]), 'negative'))
    elif test_sentiment[s] == 'neutral':
        ttweets2.append((process_tweet(test_tweets[s]), 'neutral'))
        ttweets3.append((process_tweet(test_tweets[s]), 'neutral'))
    elif test_sentiment[s] == 'positive':
        ttweets1.append((process_tweet(test_tweets[s]), 'positive'))
        ttweets3.append((process_tweet(test_tweets[s]), 'positive'))


print("full tweets: ", len(full_tweets))

print("Positive vs negative tweets (training): ", len(tweets1))
print("Positive vs negative tweets (testing): ", len(ttweets1))

print("Neutral vs negative tweets (training): ", len(tweets2))
print("Neutral vs negative tweets (testing): ", len(ttweets2))

print("Positive vs neutral tweets (training): ", len(tweets3))
print("Positive vs neutral tweets (testing): ", len(ttweets2))

#Ltrain_samples1 = int(len(tweets1)*train_percentage)
#Ltrain_samples2 = int(len(tweets2)*train_percentage)
#Ltrain_samples3 = int(len(tweets3)*train_percentage)

word_features1 = get_word_features(get_words_in_tweets(tweets1))#tweets1[0:Ltrain_samples1-1]
word_features2 = get_word_features(get_words_in_tweets(tweets2))#tweets2[0:Ltrain_samples2-1]
word_features3 = get_word_features(get_words_in_tweets(tweets3))#tweets3[0:Ltrain_samples3-1]

training_set1 = nltk.classify.apply_features(extract_features1, tweets1)#[0:Ltrain_samples1-1]
training_set2 = nltk.classify.apply_features(extract_features2, tweets2)#[0:Ltrain_samples2-1]
training_set3 = nltk.classify.apply_features(extract_features3, tweets3)#[0:Ltrain_samples3-1]
time1=time()
classifier1 = nltk.classify.SklearnClassifier(LinearSVC()).train(training_set1)
classifier2 = nltk.classify.SklearnClassifier(LinearSVC()).train(training_set2)
classifier3 = nltk.classify.SklearnClassifier(LinearSVC()).train(training_set3)
time2=time()
print("Training time: ", time2-time1)
testing_set1 = nltk.classify.apply_features(extract_features1, ttweets1)#tweets1[Ltrain_samples1:]
testing_set2 = nltk.classify.apply_features(extract_features2, ttweets2)#tweets2[Ltrain_samples2:]
testing_set3 = nltk.classify.apply_features(extract_features3, ttweets3)#tweets3[Ltrain_samples3:]

final_test = ttweets1 + ttweets2 + ttweets3#final_test = tweets1[Ltrain_samples1:] + tweets2[Ltrain_samples2:] + tweets3[Ltrain_samples3:]

print("Classifier 1 training accuracy: ",(nltk.classify.accuracy(classifier1, training_set1))*100)#training accuracy of the positive vs negative classifier
acc = [0, 0, 0]#vector of testing accuracies, will be used in voter function
acc[0]=nltk.classify.accuracy(classifier1, testing_set1)
print("Classifier 1 testing accuracy: ",(acc[0]*100))#testing accuracy of the positive vs negative classifier

print("Classifier 2 training accuracy: ",(nltk.classify.accuracy(classifier2, training_set2))*100)
acc[1]=nltk.classify.accuracy(classifier2, testing_set2)
print("Classifier 2 testing accuracy: ",(acc[1]*100))

print("Classifier 3 training accuracy: ",(nltk.classify.accuracy(classifier3, training_set3))*100)
acc[2]=nltk.classify.accuracy(classifier3, testing_set3)
print("Classifier 3 testing accuracy: ",(acc[2]*100))

def voter(tweet_test, c1, c2, c3, acc):#this voting function will give the final classification
  prediction = ['0', '0', '0']
  prediction[0] = c1.classify(extract_features1((tweet_test)))
  prediction[1] = c2.classify(extract_features2((tweet_test)))
  prediction[2] = c3.classify(extract_features3((tweet_test)))
  if prediction[0] == prediction[1]:  #those ifs look if two classifiers are giving the same result. In that case (2/3 voting same category), that will be the output.
    return prediction[0]
  elif prediction[0] == prediction[2]:
    return prediction[0]
  elif prediction[1] == prediction[2]:
    return prediction[1]
  else:
    return prediction[acc.index(max(acc))]    #if any classifier coincides, then it means that all 3 give a different class. In that case, the chosen one will be that of the classifier with more accuracy.


full tweets:  10768
Positive vs negative tweets (training):  6634
Positive vs negative tweets (testing):  789
Neutral vs negative tweets (training):  6685
Neutral vs negative tweets (testing):  855
Positive vs neutral tweets (training):  3215
Positive vs neutral tweets (testing):  855
Training time:  202.2280502319336
Classifier 1 training accuracy:  99.92463069038287
Classifier 1 testing accuracy:  80.6083650190114
Classifier 2 training accuracy:  99.3866866118175
Classifier 2 testing accuracy:  75.55555555555556
Classifier 3 training accuracy:  99.50233281493001
Classifier 3 testing accuracy:  74.2603550295858


In [ ]:
final_test = ttweets1 + ttweets2 + ttweets3   #with this function we obtain the final accuracy that takes into account the voting.
#final_test = tweets1[Ltrain_samples1:] + tweets2[Ltrain_samples2:] + tweets3[Ltrain_samples3:]
correct_counter = 0
for t in final_test:
  v = voter(t[0], classifier1, classifier2, classifier3, acc)
  if v == t[-1]:
    correct_counter = correct_counter + 1
    #print(v, " ", t[-1],": correct")
  #else:
    #print(v, " ", t[-1],": incorrect")
final_acc = (correct_counter/len(final_test))*100
print("Final accuracy: ", final_acc, " %")


Final accuracy:  65.94827586206897  %


In [ ]:
tweet = "Service it's ok"
print(voter(tweet,classifier1, classifier2, classifier3, acc))

neutral
